## Noise Experiment Setup

In [1]:
# Libraries 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM
from tensorflow.keras.optimizers import Adam

In [2]:
# Load 'ECGData.mat'
data = loadmat('ECGData.mat')
ecg_data = data['ECGData']
ecg_data_item = ecg_data[0, 0]

# Extract 'Data' and 'Labels'
Signals = ecg_data_item['Data']
Labels_array = ecg_data_item['Labels']

Labels = [Labels_array[i,0][0] for i in range(Labels_array.shape[0])]

# Map labels
Labels_mapped = []
for label in Labels:
    if label == 'NSR':
        Labels_mapped.append('N')
    elif label in ['ARR', 'CHF']:
        Labels_mapped.append('A')
    else:
        Labels_mapped.append('Other')

Labels_mapped = np.array(Labels_mapped)

# Normalize signals
Signals_normalized = []
for signal in Signals:
    signal = signal.astype(np.float32) 
    signal = (signal - np.mean(signal)) / np.std(signal)
    Signals_normalized.append(signal)
Signals_normalized = np.array(Signals_normalized)

# Encode labels
le = LabelEncoder()
segment_labels_encoded = le.fit_transform(Labels_mapped)

## Test Without Noise

In [7]:
# For demonstration, let's proceed with normal noise. 
Signals_final = Signals_normalized  # Change to Signals_noisy_uniform or Signals_noisy_exponential to test others

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    Signals_final, segment_labels_encoded, test_size=0.2, random_state=42, stratify=segment_labels_encoded)

# Reshape for RNN/LSTM: [samples, timesteps, features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

### RNN Model ###
rnn_model = Sequential()
rnn_model.add(SimpleRNN(64, input_shape=(65536, 1), activation='tanh'))
rnn_model.add(Dense(1, activation='sigmoid'))

rnn_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
rnn_history = rnn_model.fit(X_train, y_train, epochs=25, batch_size=64, validation_data=(X_test, y_test))

### LSTM Model ###
lstm_model = Sequential()
lstm_model.add(LSTM(64, input_shape=(65536, 1), activation='tanh'))
lstm_model.add(Dense(1, activation='sigmoid'))

lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
lstm_history = lstm_model.fit(X_train, y_train, epochs=25, batch_size=64, validation_data=(X_test, y_test))

# Evaluate
rnn_loss, rnn_accuracy = rnn_model.evaluate(X_test, y_test)
print(f'RNN Test Accuracy without Noise: {rnn_accuracy:.4f}')

lstm_loss, lstm_accuracy = lstm_model.evaluate(X_test, y_test)
print(f'LSTM Test Accuracy without Noise: {lstm_accuracy:.4f}')


Epoch 1/25


/Users/macbookpro2017/project research/.venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step - accuracy: 0.5565 - loss: 0.6844 - val_accuracy: 0.5758 - val_loss: 0.6559
Epoch 2/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.6810 - loss: 0.6481 - val_accuracy: 0.7879 - val_loss: 0.6201
Epoch 3/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.6926 - loss: 0.6206 - val_accuracy: 0.7273 - val_loss: 0.5968
Epoch 4/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step - accuracy: 0.6808 - loss: 0.6093 - val_accuracy: 0.7879 - val_loss: 0.5403
Epoch 5/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step - accuracy: 0.7724 - loss: 0.5598 - val_accuracy: 0.7879 - val_loss: 0.5128
Epoch 6/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step - accuracy: 0.7899 - loss: 0.5316 - val_accuracy: 0.7879 - val_loss: 0.5053
Epoch 7/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.7588 - loss: 0.5514 - val_accuracy: 0.7576 - val_loss: 0.5311
Epoch 8/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.7588 - loss: 0.5554 - val_accuracy: 0.8182 - val_loss: 0.5229
Epoch 9/25
3/3 ━━━━━━━━━━━━

### Noise Functions Definition

In [ ]:
def add_normal_noise(signals, noise_factor=0.1):
    # noise_factor: fraction of the standard deviation of the signal
    noisy_signals = []
    for sig in signals:
        sigma = np.std(sig)
        noise = np.random.normal(0, sigma * noise_factor, sig.shape)
        noisy_signals.append(sig + noise)
    return np.array(noisy_signals)

def add_uniform_noise(signals, noise_factor=0.1):
    # noise_factor: fraction of the standard deviation of the signal
    noisy_signals = []
    for sig in signals:
        sigma = np.std(sig)
        # Range of uniform noise: [-sigma*noise_factor, sigma*noise_factor]
        noise = np.random.uniform(-sigma * noise_factor, sigma * noise_factor, sig.shape)
        noisy_signals.append(sig + noise)
    return np.array(noisy_signals)

def add_exponential_noise(signals, noise_factor=0.1):
    # For exponential noise, we take an exponential distribution and adjust by noise_factor
    noisy_signals = []
    for sig in signals:
        sigma = np.std(sig)
        # scale for exponential distribution can be sigma * noise_factor
        noise = np.random.exponential(sigma * noise_factor, sig.shape)
        # Exponential is always positive; we can center it by subtracting its mean to make it more symmetric
        noise = noise - np.mean(noise)
        noisy_signals.append(sig + noise)
    return np.array(noisy_signals)

## Noise Normal

In [ ]:
Signals_noisy_normal = add_normal_noise(Signals_normalized, noise_factor=0.05)

# For demonstration, let's proceed with normal noise. 
Signals_final = Signals_noisy_normal  # Change to Signals_noisy_uniform or Signals_noisy_exponential to test others

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    Signals_final, segment_labels_encoded, test_size=0.2, random_state=42, stratify=segment_labels_encoded)

# Reshape for RNN/LSTM: [samples, timesteps, features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

### RNN Model ###
rnn_model = Sequential()
rnn_model.add(SimpleRNN(64, input_shape=(65536, 1), activation='tanh'))
rnn_model.add(Dense(1, activation='sigmoid'))

rnn_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
rnn_history = rnn_model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

### LSTM Model ###
lstm_model = Sequential()
lstm_model.add(LSTM(64, input_shape=(65536, 1), activation='tanh'))
lstm_model.add(Dense(1, activation='sigmoid'))

lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
lstm_history = lstm_model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# Evaluate
rnn_loss, rnn_accuracy = rnn_model.evaluate(X_test, y_test)
print(f'RNN Test Accuracy with Noise: {rnn_accuracy:.4f}')

lstm_loss, lstm_accuracy = lstm_model.evaluate(X_test, y_test)
print(f'LSTM Test Accuracy with Noise: {lstm_accuracy:.4f}')


Epoch 1/5


/Users/macbookpro2017/project research/.venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.4961 - loss: 0.7019 - val_accuracy: 0.8182 - val_loss: 0.6246
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.6965 - loss: 0.6499 - val_accuracy: 0.7879 - val_loss: 0.6039
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.6771 - loss: 0.6317 - val_accuracy: 0.7273 - val_loss: 0.5952
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.6498 - loss: 0.6264 - val_accuracy: 0.7879 - val_loss: 0.5541
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.7315 - loss: 0.5640 - val_accuracy: 0.8182 - val_loss: 0.5082
Epoch 1/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 33s 10s/step - accuracy: 0.5992 - loss: 0.6877 - val_accuracy: 0.6970 - val_loss: 0.6757
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 32s 9s/step - accuracy: 0.6342 - loss: 0.6728 - val_accuracy: 0.6364 - val_loss: 0.6657
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 37s 11s/step - accuracy: 0.6518 - loss: 0.6602 - val_accuracy: 0.6667 - val_loss: 0.6574
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━

## Noise Uniform

In [ ]:
Signals_noisy_uniform = add_uniform_noise(Signals_normalized, noise_factor=0.05)

# For demonstration, let's proceed with normal noise. 
Signals_final = Signals_noisy_uniform  # Change to Signals_noisy_uniform or Signals_noisy_exponential to test others

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    Signals_final, segment_labels_encoded, test_size=0.2, random_state=42, stratify=segment_labels_encoded)

# Reshape for RNN/LSTM: [samples, timesteps, features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

### RNN Model ###
rnn_model = Sequential()
rnn_model.add(SimpleRNN(64, input_shape=(65536, 1), activation='tanh'))
rnn_model.add(Dense(1, activation='sigmoid'))

rnn_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
rnn_history = rnn_model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

### LSTM Model ###
lstm_model = Sequential()
lstm_model.add(LSTM(64, input_shape=(65536, 1), activation='tanh'))
lstm_model.add(Dense(1, activation='sigmoid'))

lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
lstm_history = lstm_model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# Evaluate
rnn_loss, rnn_accuracy = rnn_model.evaluate(X_test, y_test)
print(f'RNN Test Accuracy with Noise: {rnn_accuracy:.4f}')

lstm_loss, lstm_accuracy = lstm_model.evaluate(X_test, y_test)
print(f'LSTM Test Accuracy with Noise: {lstm_accuracy:.4f}')

Epoch 1/5


/Users/macbookpro2017/project research/.venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.4630 - loss: 0.7021 - val_accuracy: 0.5455 - val_loss: 0.6756
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.6109 - loss: 0.6701 - val_accuracy: 0.6061 - val_loss: 0.6639
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.6771 - loss: 0.6499 - val_accuracy: 0.6667 - val_loss: 0.6543
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.6615 - loss: 0.6324 - val_accuracy: 0.6970 - val_loss: 0.6257
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step - accuracy: 0.7219 - loss: 0.5968 - val_accuracy: 0.7879 - val_loss: 0.5697
Epoch 1/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 36s 10s/step - accuracy: 0.4591 - loss: 0.6920 - val_accuracy: 0.7879 - val_loss: 0.6822
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 39s 12s/step - accuracy: 0.7763 - loss: 0.6773 - val_accuracy: 0.7879 - val_loss: 0.6706
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 38s 11s/step - accuracy: 0.7802 - loss: 0.6634 - val_accuracy: 0.7879 - val_loss: 0.6576
Epoch 4/5
3/3 ━━━━━━━━━━━━━━

## Noise Exponential

In [ ]:
Signals_noisy_exponential = add_exponential_noise(Signals_normalized, noise_factor=0.05)

# For demonstration, let's proceed with normal noise. 
Signals_final = Signals_noisy_exponential  # Change to Signals_noisy_uniform or Signals_noisy_exponential to test others

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    Signals_final, segment_labels_encoded, test_size=0.2, random_state=42, stratify=segment_labels_encoded)

# Reshape for RNN/LSTM: [samples, timesteps, features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

### RNN Model ###
rnn_model = Sequential()
rnn_model.add(SimpleRNN(64, input_shape=(65536, 1), activation='tanh'))
rnn_model.add(Dense(1, activation='sigmoid'))

rnn_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
rnn_history = rnn_model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

### LSTM Model ###
lstm_model = Sequential()
lstm_model.add(LSTM(64, input_shape=(65536, 1), activation='tanh'))
lstm_model.add(Dense(1, activation='sigmoid'))

lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
lstm_history = lstm_model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# Evaluate
rnn_loss, rnn_accuracy = rnn_model.evaluate(X_test, y_test)
print(f'RNN Test Accuracy with Noise: {rnn_accuracy:.4f}')

lstm_loss, lstm_accuracy = lstm_model.evaluate(X_test, y_test)
print(f'LSTM Test Accuracy with Noise: {lstm_accuracy:.4f}')